# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
!pip install stable-baselines3[extra] protobuf==3.20.*

In [ ]:
!pip install mss pydirectinput

In [ ]:
#chrome://dino/

In [ ]:
# import libraries


# 2. Build the Environment

# 2.0.1 Detect the Dino game Region

In [ ]:
#detect the screen position (left, top, right, bottom)


## 2.1 Create Environment

In [ ]:
class WebGame(Env):
    def __init__(self):
        super().__init__()
        # Setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,83,150), dtype=np.uint8)
        self.action_space = Discrete(3)
        # Capture game frames
        self.cap = mss()
         # Modify the game region based on the above result
        self.game_location = {'left': 85, 'top': 280, 'width': 500, 'height': 190}
         # Modify the game over region based on the above result
        self.game_over_location = {'left': 290, 'top': 280, 'width': 380, 'height': 40}
        
        
    def step(self, action):
        action_map = {
            0:'space',
            1: 'down', 
            2: 'no_op'
        }
        if action !=2:
            pydirectinput.press(action_map[action])

        game_over, game_over_cap = self.is_game_over() 
        observation = self.get_observation()
        reward = 1 
        truncated = False
        info = {}
        return observation, reward, game_over, truncated, info
        
    
    def reset(self, seed=None):
        time.sleep(1)
        pydirectinput.click(x=150, y=150)
        pydirectinput.press('space')
        return self.get_observation(), {}
             
    def close(self):
        cv2.destroyAllWindows()
    
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (150,83))
        channel = np.reshape(resized, (1,83,150))
        return channel
    
    def is_game_over(self):
        game_over_cap = np.array(self.cap.grab(self.game_over_location))
        game_over=False
        if np.mean(game_over_cap) > 95 and np.mean(game_over_cap) < 105:
            game_over=True
        return game_over, game_over_cap

# 2.2 Test Environment

In [ ]:
# Create the WebGame environment


In [ ]:
# Get the observation for the game location region


In [ ]:
# Check the game region by visual


In [ ]:
# Check if the game over status and game over region


In [ ]:
# Print out the game over status and image


In [ ]:
# Randomly play the game
# Change a value such that it plays 5 times.
for episode in range(3): 
    obs = env.reset()
    game_over = False  
    total_reward = 0
    while not game_over: 
        obs, reward, game_over, truncated, info =  env.step(env.action_space.sample())
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))

# 3. Train the Model

## 3.1 Create Callback

In [ ]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker

In [ ]:
# Let the RL to test the environment


In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
# Define two directories: train and logs


In [ ]:
# Create callback which include check_freq=1000 and save_path=CHECKPOINT_DIR


## 3.2 Build DQN and Train

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [ ]:
# Create WebGame Environment



In [ ]:
# Define a RL model


In [ ]:
# In the class, set the steps to 10000 to confirm the RL running correctly
# To see a good RL result, you need around 100,000 steps

# Ctrl-shift-p and select "launch TensorBoard"-> select log folder


# 4. Test out Model

In [ ]:
# Load a trained model
model.load("train\\best_model_10000.zip") 

In [ ]:
# Use the model to play the game for 3 times


# 5. Upload to github Repo and publish it publicly